# Predicción del aterrizaje de la primera etapa del Falcon 9 de SpaceX

## Laboratorio 2: Manipulación de datos
En este laboratorio, realizaremos un análisis exploratorio de datos (EDA) para encontrar algunos patrones en los datos y determinar cuál sería la etiqueta para entrenar modelos supervisados.

En el conjunto de datos, hay varios casos diferentes en los que el cohete no aterrizó con éxito. A veces, se intentó un aterrizaje pero falló debido a un accidente; por ejemplo, <code>True Ocean</code> significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que <code>False Ocean</code> significa que el resultado de la misión aterrizó sin éxito en una región específica del océano. <code>True RTLS</code> significa que el resultado de la misión aterrizó con éxito en una plataforma de tierra. <code>False RTLS</code> significa que el resultado de la misión aterrizó sin éxito en una plataforma de tierra. <code>True ASDS</code> significa que el resultado de la misión aterrizó con éxito en un barco no tripulado. <code>False ASDS</code> significa que el resultado de la misión aterrizó sin éxito en un barco no tripulado.

En este laboratorio, convertiremos principalmente esos resultados en etiquetas de entrenamiento, donde “1” significa que el amplificador aterrizó exitosamente y “0” significa que no tuvo éxito.

### Objetivos
Realizar análisis exploratorio de datos y determinar etiquetas de entrenamiento

- Análisis exploratorio de datos
- Determinar etiquetas de entrenamiento

In [1]:
import pandas as pd
import numpy as np

### Análisis de datos
Conjunto de datos del espacio de carga X, de la sección anterior.

In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identificar y calcular el porcentaje de valores faltantes en cada atributo

In [3]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identifique qué columnas son numéricas y categóricas:

In [4]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TAREA 1: Calcular el número de lanzamientos en cada sitio
Los datos contienen varias instalaciones de lanzamiento de Space X: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Complejo de lanzamiento espacial de Cabo Cañaveral</a> 40 <b>VAFB SLC 4E </b>, Complejo de lanzamiento espacial 4E de la Base Aérea Vandenberg <b>(SLC-4E)</b>, Complejo de lanzamiento 39A del Centro Espacial Kennedy <b>KSC LC 39A </b>. La ubicación de cada lanzamiento se coloca en la columna <code>LaunchSite</code>

A continuación, veamos la cantidad de lanzamientos de cada sitio.

Utilice el método <code>value_counts()</code> en la columna <code>LaunchSite</code> para determinar la cantidad de lanzamientos en cada sitio:


In [5]:
# Aplicar value_counts() en la columna LaunchSite
df.value_counts('LaunchSite')

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Cada lanzamiento apunta a una órbita dedicada, y estos son algunos tipos de órbitas comunes:

* <b>LEO</b>: La órbita terrestre baja (LEO) es una órbita centrada en la Tierra con una altitud de 2000 km (1200 mi) o menos (aproximadamente un tercio del radio de la Tierra),[1] o con al menos 11,25 períodos por día (un período orbital de 128 minutos o menos) y una excentricidad menor a 0,25.[2] La mayoría de los objetos creados por el hombre en el espacio exterior están en LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Las órbitas terrestres muy bajas (VLEO) se pueden definir como las órbitas con una altitud media inferior a 450 km. Operar en estas órbitas puede proporcionar una serie de beneficios a las naves espaciales de observación de la Tierra, ya que la nave espacial opera más cerca del punto de observación<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.

* <b>GTO</b> Una órbita geoestacionaria es una órbita terrestre alta que permite a los satélites adaptarse a la rotación de la Tierra. Ubicada a 22,236 millas (35,786 kilómetros) sobre el ecuador de la Tierra, esta posición es un punto valioso para monitorear el clima, las comunicaciones y la vigilancia. Debido a que el satélite orbita a la misma velocidad que gira la Tierra, el satélite parece permanecer en su lugar a lo largo de una única longitud, aunque puede desplazarse de norte a sur”, escribió la NASA en su sitio web Earth Observatory <a href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.

* <b>SSO (o SO)</b>: Es una órbita heliosincrónica, también llamada órbita heliosincrónica, es una órbita casi polar alrededor de un planeta, en la que el satélite pasa sobre cualquier punto dado de la superficie del planeta a la misma hora solar media local <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.

* <b>ES-L1</b>: En los puntos de Lagrange, las fuerzas gravitacionales de los dos cuerpos grandes se cancelan de tal manera que un objeto pequeño colocado en órbita allí está en equilibrio en relación con el centro de masa de los cuerpos grandes. L1 es uno de esos puntos entre el Sol y la Tierra <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .

* <b>HEO</b> Una órbita altamente elíptica es una órbita elíptica con alta excentricidad, generalmente se refiere a una órbita alrededor de la Tierra <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.

* <b> ISS </b> Una estación espacial modular (satélite artificial habitable) en órbita terrestre baja. Se trata de un proyecto colaborativo multinacional entre cinco agencias espaciales participantes: NASA (Estados Unidos), Roscosmos (Rusia), JAXA (Japón), ESA (Europa) y CSA (Canadá). <a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>

* <b> MEO </b> Órbitas geocéntricas que varían en altitud desde los 2000 km (1200 mi) hasta justo por debajo de la órbita geosincrónica a 35 786 kilómetros (22 236 mi). También conocida como órbita circular intermedia. Estos se encuentran "más comúnmente a 20.200 kilómetros (12.600 mi), o 20.650 kilómetros (12.830 mi), con un período orbital de 12 horas <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>

* <b> HEO </b> Órbitas geocéntricas por encima de la altitud de la órbita geosincrónica (35.786 km o 22.236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>

* <b> GEO </b> Es una órbita geosincrónica circular a 35.786 kilómetros (22.236 millas) sobre el ecuador de la Tierra y que sigue la dirección de la rotación de la Tierra <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>

* <b> PO </b> Es un tipo de satélite en el que un satélite pasa por encima o casi por encima de ambos polos del cuerpo que orbita (normalmente un planeta como la Tierra <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

algunos se muestran en el siguiente gráfico:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)

### TAREA 2: Calcular el número y ocurrencia de cada órbita
Utilice el método <code>.value_counts()</code> para determinar el número y la ocurrencia de cada órbita en la columna <code>Orbit</code>

In [6]:
# Aplicar value_counts en la columna Orbit
df.value_counts('Orbit')

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
GEO       1
HEO       1
SO        1
Name: count, dtype: int64

### TAREA 3: Calcular el número y la ocurrencia del resultado de la misión de las órbitas.
Utilice el método <code>.value_counts()</code> en la columna <code>Outcome</code> para determinar el número de <code>landing_outcomes</code>. Luego asígnelo a una variable landing_outcomes.

In [7]:
# landing_outcomes = valores en la columna Resultado
landing_outcomes = df.value_counts('Outcome')
print(landing_outcomes)

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64


<code>True Ocean</code> significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que <code>False Ocean</code> significa que el resultado de la misión aterrizó sin éxito en una región específica del océano. <code>True RTLS</code> significa que el resultado de la misión aterrizó con éxito en una plataforma de tierra. <code>False RTLS</code> significa que el resultado de la misión aterrizó sin éxito en una plataforma de tierra. <code>True ASDS</code> significa que el resultado de la misión aterrizó con éxito en un barco no tripulado. <code>False ASDS</code> significa que el resultado de la misión aterrizó sin éxito en un barco no tripulado. <code>None ASDS</code> y <code>None None</code> representan un aterrizaje fallido.

In [8]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


Creamos un conjunto de resultados donde la segunda etapa no aterrizó con éxito:

In [9]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### TAREA 4: Crear una etiqueta de resultado de aterrizaje a partir de la columna Resultado
Utilizando <code>Outcome</code>, crea una lista donde el elemento sea cero si la fila correspondiente en <code>Outcome</code> está en el conjunto <code>bad_outcome</code>; de lo contrario, es uno. Luego asígnalo a la variable <code>landing_class</code>:

In [10]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise

# Obtener los valores de Outcome y bad_outcomes
landing_outcomes = df['Outcome'].value_counts()
bad_outcomes = set(landing_outcomes.keys()[[1, 3, 5, 6, 7]])

# Crear la lista landing_class
landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]

# Asignar la lista a la columna 'LandingClass' del DataFrame
df['LandingClass'] = landing_class

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())


   FlightNumber        Date BoosterVersion  PayloadMass Orbit    LaunchSite  \
0             1  2010-06-04       Falcon 9  6104.959412   LEO  CCAFS SLC 40   
1             2  2012-05-22       Falcon 9   525.000000   LEO  CCAFS SLC 40   
2             3  2013-03-01       Falcon 9   677.000000   ISS  CCAFS SLC 40   
3             4  2013-09-29       Falcon 9   500.000000    PO   VAFB SLC 4E   
4             5  2013-12-03       Falcon 9  3170.000000   GTO  CCAFS SLC 40   

       Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  \
0    None None        1     False   False  False        NaN    1.0   
1    None None        1     False   False  False        NaN    1.0   
2    None None        1     False   False  False        NaN    1.0   
3  False Ocean        1     False   False  False        NaN    1.0   
4    None None        1     False   False  False        NaN    1.0   

   ReusedCount Serial   Longitude   Latitude  LandingClass  
0            0  B0003  -80.577366  28.56185

Esta variable representará la variable de clasificación que representa el resultado de cada lanzamiento. Si el valor es cero, la primera etapa no aterrizó con éxito; uno significa que la primera etapa aterrizó con éxito

In [11]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [12]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,LandingClass,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0,0


Podemos utilizar la siguiente línea de código para determinar la tasa de éxito:

In [13]:
df["Class"].mean()

0.6666666666666666

Ahora podemos exportarlo a un CSV para la siguiente sección, pero para que las respuestas sean consistentes, en el próximo laboratorio proporcionaremos datos en un rango de fechas preseleccionado.

In [14]:
df.to_csv("dataset_part_2.csv", index=False)